## Project Overview

### Problem Statement
In today's digital world, understanding how Reddit communities function is crucial for moderators, users, and researchers...

### Data Collection Overview
- **Tools**:
  - PRAW (Python Reddit API Wrapper)
  - BeautifulSoup / Scrapy
  
- **Data Points to Collect**:
  - **Posts**: Title, content, upvotes...
  - **Comments**: Content, upvotes...
  
### Solution Approach
1. **Sentiment & Engagement Analysis**
   - Visualizations using Matplotlib and Seaborn...
   
2. **Correlation Analysis**
   - Apply classification algorithms using scikit-learn...

### Expected Deliverables
- Insight Report
- Actionable Recommendations

## Setup Environment

### Purpose
This section prepares our Google Colab environment for the Reddit Communities analysis project as outlined in our team's proposal. We'll install the necessary Python libraries to handle data collection, processing, analysis, and visualization.

### Key Libraries
- PRAW: For accessing the Reddit API
- pandas: For data manipulation and analysis
- numpy: For numerical computing
- matplotlib and seaborn: For data visualization
- nltk: For natural language processing and sentiment analysis
- scikit-learn: For machine learning tasks

### Alignment with Project Goals
These libraries support our objectives of:
1. Analyzing moderation strategies
2. Predicting post impact
3. Visualizing Reddit community interactions

### Installation Code
Run the following cell to install the required libraries:


In [33]:
import numpy as np
import pandas as pd
import matplotlib
import seaborn
import nltk
%pip install praw
import praw
from datetime import datetime


Note: you may need to restart the kernel to use updated packages.


# Praw API Config

This block of code create tje connection with the reddit application and by authenticating.

In [34]:
def setup_reddit_api():
    return praw.Reddit(
        client_id="nrakGjG_wnBE_5UdcHNJoQ",
        client_secret="qmGr1q_4pGIBR0pYJE8cyhUbTbdX2w",
        user_agent="LittleCheesyExplorers/1.0 (Reddit Communities Analysis Project)"
    )

reddit = setup_reddit_api()

print(reddit.user.me())
subreddit = reddit.subreddit("science")
print(subreddit.display_name)
print(subreddit.title)

None
science
Reddit Science


# Read required Subreddits 
This code block will store the names of the subreddits that we want to collect data from.

The names will be stored in a text file and we will read from that and scrape based on that list.

In [35]:
# Read subreddit names from the text file
with open('subreddits.txt', 'r') as file:
    subreddit_list = [line.strip() for line in file if line.strip()]

print(subreddit_list)

['AskReddit', 'news', 'funny', 'gaming', 'todayilearned', 'science']


# Collection Methods

In [36]:
def collect_subreddit_posts(subreddit_name, post_limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    posts_data = []

    for post in subreddit.hot(limit=post_limit):
        posts_data.append({
            'subreddit': subreddit_name,
            'title': post.title,
            'content': post.selftext,
            'upvotes': post.score,
            'upvote_ratio': post.upvote_ratio,
            'comments_count': post.num_comments,
            'author': post.author.name if post.author else '[deleted]',
            'timestamp': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
            'post_id': post.id
        })

    return pd.DataFrame(posts_data)

def collect_posts_from_subreddits(subreddit_list, post_limit=10):
    all_posts = []
    
    for subreddit_name in subreddit_list:
        print(f"Collecting posts from r/{subreddit_name}")
        try:
            posts_df = collect_subreddit_posts(subreddit_name, post_limit)
            all_posts.append(posts_df)
            print(f"Collected {len(posts_df)} posts from r/{subreddit_name}")
        except Exception as e:
            print(f"Error collecting posts from r/{subreddit_name}: {str(e)}")
    
    combined_df = pd.concat(all_posts, ignore_index=True)
    return combined_df

# Running Collection Methods

In [3]:
# Collect posts from all subreddits
all_posts_df = collect_posts_from_subreddits(subreddit_list, post_limit=10000)

# Save all collected posts to a single CSV file
csv_filename = "subreddit_posts.csv"
all_posts_df.to_csv(csv_filename, index=False)
print(f"\nAll posts data saved to {csv_filename}")

# Print summary of collected posts
print("\nSummary of collected posts:")
print(all_posts_df['subreddit'].value_counts())

# Sample data output
print("\nSample post data:")
print(all_posts_df.iloc[0] if len(all_posts_df) > 0 else "No posts collected")

NameError: name 'collect_posts_from_subreddits' is not defined

In [ ]:
# adding derived columns 

import pandas as pd
from datetime import datetime

# Load the existing CSV file
df = pd.read_csv('subreddit_posts.csv')

# Calculate Title Length
df['title_length'] = df['title'].apply(len)

# Calculate Post Length (use content column, assuming it's a text-based post)
df['post_length'] = df['content'].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)

# Convert the timestamp to  datetime format
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

# Calculate Time of Day (hour of the day)
df['time_of_day'] = df['timestamp'].dt.hour

# Calculate Day of the Week
df['day_of_week'] = df['timestamp'].dt.day_name()

# Save the updated DataFrame to a new CSV file
df.to_csv('subreddit_posts.csv', index=False)

print("New features added and file saved successfully.")


New features added and file saved successfully.


# CSV Creation and Storing
In this Code Block we will be creating a csv file and storing all our scraped data.